In [4]:
import os
import sys

import torch 
import torch.nn as nn

In [5]:
def generate(model, tokenizer, prompt, max_length=50, temperature=0.7) -> str:
    """
    Generate text from a prompt using the model
    """
    tokens = tokenizer.encode(prompt)
    tokens = torch.Tensor(tokens).long().unsqueeze(0)
    for _ in range(max_length):
        logits = model(tokens)[0][:, -1, :]
        probs = torch.softmax(logits / temperature, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)
        tokens = torch.cat([tokens, next_token], dim=-1)
    return tokenizer.decode_sequence(tokens[0].tolist())

In [8]:
#sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "..",)))
#print("\n".join(sys.path))
# %%
current_dir = os.path.dirname(os.path.abspath("spar_sae_circuit_sandbox.ipynb"))
model_dir = os.path.join(current_dir, '..) # Assuming it's one level up
#toy_model_dir = os.path.join(current_dir, '..', 'llm_from_scratch/LLM_from_scratch/')

sys.path.append(model_dir)
#sys.path.append(toy_model_dir)

from config.gpt.training import options
from models.gpt import GPT
from data.tokenizers import ASCIITokenizer, TikTokenTokenizer

from utils import generate
name = 'shakespeare_64x4'
config = options[name]

model = GPT(config.gpt_config)
model_path = os.path.join("../checkpoints", name)
model = model.load(model_path, device=config.device)

tokenizer = ASCIITokenizer() if "shake" in name else TikTokenTokenizer()

print(generate(model, tokenizer, "Today I thought,", max_length=100))

ModuleNotFoundError: No module named 'models'